In [2]:
import requests
import pandas as pd
import json
import plotly.io as pio
import plotly.express as px

In [82]:
villes=["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",]
# "Colmar",
# "Eguisheim",
# "Besancon",
# "Dijon",
# "Annecy",
# "Grenoble",
# "Lyon",
# "Gorges du Verdon",
# "Bormes les Mimosas",
# "Cassis",
# "Marseille",
# "Aix en Provence",]
# "Avignon",
# "Uzes",
# "Nimes",
# "Aigues Mortes",
# "Saintes Maries de la mer",
# "Collioure",
# "Carcassonne",
# "Ariege",
# "Toulouse",
# "Montauban",
# "Biarritz",
# "Bayonne",
# "La Rochelle"]

# ***API calls per day are limited  ***

In [83]:
rez=[]
for i in range(len(villes)):
  r=requests.get("https://nominatim.openstreetmap.org/search?",params={'city':villes[i],'format':'json','country':'france'})
  rez.append(r.json())

In [84]:
rez

[[{'place_id': 156604934,
   'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
   'osm_type': 'way',
   'osm_id': 211285890,
   'boundingbox': ['48.6349172', '48.637031', '-1.5133292', '-1.5094796'],
   'lat': '48.6359541',
   'lon': '-1.511459954959514',
   'display_name': 'Mont Saint-Michel, Le Mont-Saint-Michel, Avranches, Manche, Normandie, France métropolitaine, 50170, France',
   'class': 'place',
   'type': 'islet',
   'importance': 0.855436556781574}],
 [{'place_id': 298187446,
   'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
   'osm_type': 'relation',
   'osm_id': 905534,
   'boundingbox': ['48.5979853', '48.6949736', '-2.0765246', '-1.9367259'],
   'lat': '48.649518',
   'lon': '-2.0260409',
   'display_name': 'Saint-Malo, Ille-et-Vilaine, Bretagne, France métropolitaine, 35400, France',
   'class': 'boundary',
   'type': 'administrative',
   'importance': 0.7764672172428952,
   'icon': 'https://nominat

# ***une liste de dic***

In [85]:
len(rez)

10

In [86]:
rez[2][0]

{'place_id': 298563984,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 145776,
 'boundingbox': ['49.2608124', '49.2934736', '-0.7275671', '-0.6757378'],
 'lat': '49.2764624',
 'lon': '-0.7024738',
 'display_name': 'Bayeux, Calvados, Normandie, France métropolitaine, 14400, France',
 'class': 'boundary',
 'type': 'administrative',
 'importance': 0.7827003853621928,
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/poi_boundary_administrative.p.20.png'}

# ***on extrait le nom de la ville et d'autres infos lon ,lat ,id.... ***

In [87]:
rez[2][0]['display_name'].split(",")[0]

'Bayeux'

# ***work for 1 work for all ***

In [74]:
dic={'names':rez[0][0]['display_name'].split(",")[0],'lat':rez[0][0]['lat'],'lon':rez[i][0]['lon']}

In [88]:
df=pd.DataFrame(dic,index=[0])
df

,names,lat,lon
0,Mont Saint-Michel,48.6359541,5.4474738


# ***on construit un dataframe contenat le nom / lat / lon de chaque ville***

In [90]:
for i in range(len(rez)):
  dic1={'names':rez[i][0]['display_name'].split(",")[0],'lat':rez[i][0]['lat'],'lon':rez[i][0]['lon']}
  df1=pd.DataFrame(dic1,index=[i])
  # df=df.append(df1)
  df = pd.concat([df, df1])
df.head()

,names,lat,lon
0,Mont Saint-Michel,48.6359541,5.4474738
-1,Mont Saint-Michel,48.6359541,-1.511459954959514
0,Saint-Malo,48.649518,-2.0260409
1,Bayeux,49.2764624,-0.7024738
2,Le Havre,49.4938975,0.1079732


In [31]:
df2=df.iloc[1:,:]
df2

,names,lat,lon
0,Mont Saint-Michel,48.6359541,-1.511459954959514
1,Saint-Malo,48.649518,-2.0260409
2,Bayeux,49.2764624,-0.7024738
3,Le Havre,49.4938975,0.1079732
4,Rouen,49.4404591,1.0939658
5,Paris,48.8534951,2.3483915
6,Amiens,49.8941708,2.2956951
7,Lille,50.6365654,3.0635282
8,Strasbourg,48.584614,7.7507127
9,Château du Haut-Kœnigsbourg,48.2495226,7.3454923


In [91]:
df2.loc[1:,['lat']]


,lat
1,48.649518
2,49.2764624
3,49.4938975
4,49.4404591
5,48.8534951
6,49.8941708
7,50.6365654
8,48.584614
9,48.2495226


# ***le nb de lignes et cols pour faire des boucles***

In [33]:
n,m=df2.shape

# ***L' API est payant apres certain nbr de requests***

In [41]:
keyhere="4c471960731aa75b945cf729eeee897a"

# ***on va travailler sur (rain et windspeed)***

In [44]:
rainlist=[]
windlinst=[]
lat=[]
lon=[]
for i in range(n):
  a=df2.loc[i,['lat']]
  b=df2.loc[i,['lon']]

  # print(a ,b)

  #les params
  paylods={'lat':a ,'lon':b ,'cnt':'7', 'exclude':'minutely','appid':keyhere}
  rweather=requests.get('https://api.openweathermap.org/data/3.0/onecall?',params=paylods)

  # print(rweather.json().keys())

  dailyinfo=pd.DataFrame(rweather.json()['daily'])
  dailyinfo=dailyinfo.loc[:,['wind_speed','rain']]

  # print(dailyinfo.mean())

  rainlist.append(dailyinfo['rain'].mean())
  windlinst.append(dailyinfo['wind_speed'].mean())
  lat.append(a)
  lon.append(b)

print('les villes',list(df2['names']))
print('rain mean',rainlist)
print('windspeed mean',windlinst)



les villes ['Mont Saint-Michel', 'Saint-Malo', 'Bayeux', 'Le Havre', 'Rouen', 'Paris', 'Amiens', 'Lille', 'Strasbourg', 'Château du Haut-Kœnigsbourg']
rain mean [1.2700000000000002, 0.8283333333333335, 1.5266666666666666, 2.16, 0.7233333333333333, 3.5549999999999997, 0.7366666666666667, 0.7933333333333333, 4.0575, 2.6275]
windspeed mean [6.55875, 7.032500000000001, 6.734999999999999, 6.992500000000001, 7.08625, 4.772499999999999, 6.95875, 7.04875, 4.82125, 4.0225]


# ***dic apres df apres csv***

In [45]:
datafinal=pd.DataFrame({'citys' : list(df2['names']) ,
                        'rain_mean' : rainlist},
                       columns=['citys','rain_mean'])
datafinal['windspeed_mean']=windlinst


In [50]:
datafinal.head(1)

,citys,rain_mean,windspeed_mean,lat,lon
0,Mont Saint-Michel,1.27,6.55875,48.6359541,-1.511459954959514


In [92]:
df2.loc[:,['lat','lon']] #to test
datafinal['lat']=df2.loc[:,['lat']]
datafinal['lon']=df2.loc[:,['lon']]
print('done')

done


# ***datafinal ready***

In [93]:
datafinal

,citys,rain_mean,windspeed_mean,lat,lon
0,Mont Saint-Michel,1.270000,6.55875,48.6359541,-1.511459954959514
1,Saint-Malo,0.828333,7.03250,48.649518,-2.0260409
2,Bayeux,1.526667,6.73500,49.2764624,-0.7024738
3,Le Havre,2.160000,6.99250,49.4938975,0.1079732
4,Rouen,0.723333,7.08625,49.4404591,1.0939658
5,Paris,3.555000,4.77250,48.8534951,2.3483915
6,Amiens,0.736667,6.95875,49.8941708,2.2956951
7,Lille,0.793333,7.04875,50.6365654,3.0635282
8,Strasbourg,4.057500,4.82125,48.584614,7.7507127
9,Château du Haut-Kœnigsbourg,2.627500,4.02250,48.2495226,7.3454923


# ***save it into csv ***

In [94]:
import os
os.makedirs('projet kayak', exist_ok=True)
datafinal.to_csv('projet kayak/kayak.csv')

In [95]:
datafinal.to_csv(index=False)

'citys,rain_mean,windspeed_mean,lat,lon\nMont Saint-Michel,1.2700000000000002,6.55875,48.6359541,-1.511459954959514\nSaint-Malo,0.8283333333333335,7.032500000000001,48.649518,-2.0260409\nBayeux,1.5266666666666666,6.734999999999999,49.2764624,-0.7024738\nLe Havre,2.16,6.992500000000001,49.4938975,0.1079732\nRouen,0.7233333333333333,7.08625,49.4404591,1.0939658\nParis,3.5549999999999997,4.772499999999999,48.8534951,2.3483915\nAmiens,0.7366666666666667,6.95875,49.8941708,2.2956951\nLille,0.7933333333333333,7.04875,50.6365654,3.0635282\nStrasbourg,4.0575,4.82125,48.584614,7.7507127\nChâteau du Haut-Kœnigsbourg,2.6275,4.0225,48.2495226,7.3454923\n'

In [96]:
test=pd.read_csv('/content/projet kayak/kayak.csv')
test.head()

,Unnamed: 0,citys,rain_mean,windspeed_mean,lat,lon
0,0,Mont Saint-Michel,1.270000,6.55875,48.635954,-1.511460
1,1,Saint-Malo,0.828333,7.03250,48.649518,-2.026041
2,2,Bayeux,1.526667,6.73500,49.276462,-0.702474
3,3,Le Havre,2.160000,6.99250,49.493898,0.107973
4,4,Rouen,0.723333,7.08625,49.440459,1.093966


# **to drop unamed 0**

In [ ]:
# test.drop('Unnamed: 0',axis=1,inplace=True)

# ***lets rename it nd keep it for now ***

In [97]:
test=test.rename(columns={"Unnamed: 0": "id"})

# ***sort by rain nd wndspd***

In [98]:
sorted=test.sort_values(by=['rain_mean','windspeed_mean'],ascending=True,axis=0)
sorted

,id,citys,rain_mean,windspeed_mean,lat,lon
4,4,Rouen,0.723333,7.08625,49.440459,1.093966
6,6,Amiens,0.736667,6.95875,49.894171,2.295695
7,7,Lille,0.793333,7.04875,50.636565,3.063528
1,1,Saint-Malo,0.828333,7.03250,48.649518,-2.026041
0,0,Mont Saint-Michel,1.270000,6.55875,48.635954,-1.511460
2,2,Bayeux,1.526667,6.73500,49.276462,-0.702474
3,3,Le Havre,2.160000,6.99250,49.493898,0.107973
9,9,Château du Haut-Kœnigsbourg,2.627500,4.02250,48.249523,7.345492
5,5,Paris,3.555000,4.77250,48.853495,2.348391
8,8,Strasbourg,4.057500,4.82125,48.584614,7.750713


# ***plotly x***

In [105]:
fig = px.scatter_mapbox(sorted,
                        lat="lat",
                        lon="lon",
                        color="rain_mean",
                        size="windspeed_mean",
                        zoom=5,
                        hover_name="citys",
                        mapbox_style="carto-positron")
fig.show()


In [106]:
pio.write_html(fig, "figure.html")

# ***api part ends here ***